In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/understanding-unets


In [2]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import time
import warnings
warnings.filterwarnings("ignore")

import bm3d
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

from learning_wavelets.data import im_generators, ConcatenateGenerators
from learning_wavelets.dncnn import dncnn
from learning_wavelets.evaluate import Metrics
from learning_wavelets.keras_utils.thresholding import SoftThresholding, HardThresholding, DynamicSoftThresholding, DynamicHardThresholding
from learning_wavelets.learned_wavelet import learned_wavelet, learnlet
from learning_wavelets.unet import unet
from learning_wavelets.wavelet_denoising import wavelet_denoising_pysap


                 .|'''|       /.\      '||'''|,
                 ||          // \\      ||   ||
'||''|, '||  ||` `|'''|,    //...\\     ||...|'
 ||  ||  `|..||   .   ||   //     \\    ||
 ||..|'      ||   |...|' .//       \\. .||
 ||       ,  |'
.||        ''

Package version: 0.0.3

License: CeCILL-B

Authors: 

Antoine Grigis <antoine.grigis@cea.fr>
Samuel Farrens <samuel.farrens@cea.fr>
Jean-Luc Starck <jl.stark@cea.fr>
Philippe Ciuciu <philippe.ciuciu@cea.fr>

Dependencies: 

scipy          : >=1.3.0   - required | 1.3.0     installed
numpy          : >=1.16.4  - required | 1.17.4    installed
matplotlib     : >=3.0.0   - required | 3.1.0     installed
astropy        : >=3.0.0   - required | 3.1.2     installed
nibabel        : >=2.3.2   - required | 2.5.1     installed
pyqtgraph      : >=0.10.0  - required | 0.10.0    installed
progressbar2   : >=3.34.3  - required | ?         installed
modopt         : >=1.4.0   - required | 1.4.1     installed
scikit-learn   : >=0.19.1  - requi

In [4]:
np.random.seed(0)

In [5]:
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['image.cmap'] = 'gray'

In [6]:
grey = True
n_channels = 3
if grey:
    n_channels = 1
noise_std = 30
im_gen_train, im_gen_val, im_gen_test, size, n_samples_train = im_generators(
    'bsd68', 
    batch_size=1, 
    validation_split=0, 
    no_augment=True, 
    noise_std=noise_std,
    grey=grey
)    
im_bsd68 = im_gen_test

In [7]:
thresh = 2*noise_std/255
all_net_params = [
#     {
#         'name': 'unet',
#         'init_function': unet,
#         'run_params': {
#             'n_layers': 5, 
#             'pool': 'max', 
#             "layers_n_channels": [64, 128, 256, 512, 1024], 
#             'layers_n_non_lins': 2,
#             'non_relu_contract': False,
#             'input_size': (size, size, n_channels),
#         },
#         'run_id': 'unet_div2k_30_1574408775',
#     },
#     {
#         'name': 'learned wavelet_st',
#         'init_function': learned_wavelet,
#         'run_params': {
#             'n_scales': 5, 
#             'n_details': 256, 
#             'n_coarse': n_channels, 
#             'mixing_details': False,
#             'denoising_activation': SoftThresholding(thresh),
#             'wav_pooling': True,
#             'wav_use_bias': False,
#             'wav_normed': True,
#             'filters_normed': ['details', 'coarse'],
#             'input_size': (size, size, n_channels),     
#         },
#         'run_id': 'learned_wavelet_div2k_30_1574790444',
#         'epoch': 250,
#     },
#     {
#         'name': 'dncnn_50',
#         'init_function': dncnn,
#         'run_params': {
#             'filters': 64, 
#             'depth': 20,
#             'input_size': (size, size, n_channels),     
#         },
#         'run_id': 'dncnn_div2k_30_1575738740',
#         'epoch': 150,
#     },
    
#     {
#         'name': 'learned wavelet_ht',
#         'init_function': learned_wavelet,
#         'run_params': {
#             'n_scales': 5, 
#             'n_details': 256, 
#             'n_coarse': n_channels, 
#             'mixing_details': False,
#             'denoising_activation': HardThresholding(1.5*thresh),
#             'wav_pooling': True,
#             'wav_use_bias': False,
#             'wav_normed': True,
#             'filters_normed': ['details', 'coarse'],
#             'input_size': (size, size, n_channels),
#         },
#         'run_id': 'learned_wavelet_div2k_30_1574371296',
#         'epoch': 50,
#     },
]
dynamic_denoising_net_params = [
    {
        'name': 'learnlet_0_55',
        'init_function': learnlet,
        'run_params': {
            'denoising_activation': DynamicSoftThresholding(2, trainable=False),
            'learnlet_analysis_kwargs':{
                'n_tiling': 256, 
                'mixing_details': False,        
            },
            'learnlet_synthesis_kwargs': {
                'res': False,
            },
            'n_scales': 5,
            'exact_reconstruction_weight': 0,
            'input_size': (size, size, n_channels),     
        },
        'run_id': 'learnlet_dynamic_st_div2k_(0, 55)_1575835369',
        'epoch': 250,
    },
    {
        'name': 'learnlet_0_55_alpha_learned',
        'init_function': learnlet,
        'run_params': {
            'denoising_activation': DynamicSoftThresholding(2, trainable=True),
            'learnlet_analysis_kwargs':{
                'n_tiling': 256, 
                'mixing_details': False,        
            },
            'learnlet_synthesis_kwargs': {
                'res': False,
            },
            'n_scales': 5,
            'exact_reconstruction_weight': 0,
            'input_size': (size, size, n_channels),     
        },
        'run_id': 'learnlet_dynamic_st_div2k_(0, 55)_1575881431',
        'epoch': 250,
    },
    {
        'name': 'learnlet_0_55_alpha_learned_bsd',
        'init_function': learnlet,
        'run_params': {
            'denoising_activation': DynamicSoftThresholding(2, trainable=True),
            'learnlet_analysis_kwargs':{
                'n_tiling': 256, 
                'mixing_details': False,        
            },
            'learnlet_synthesis_kwargs': {
                'res': False,
            },
            'n_scales': 5,
            'exact_reconstruction_weight': 0,
            'input_size': (size, size, n_channels),     
        },
        'run_id': 'learnlet_dynamic_st_bsd500_0_55_1575932444',
        'epoch': 250,
    },
    {
        'name': 'learnlet_0_55_ht',
        'init_function': learnlet,
        'run_params': {
            'denoising_activation': DynamicHardThresholding(3, trainable=False),
            'learnlet_analysis_kwargs':{
                'n_tiling': 256, 
                'mixing_details': False,        
            },
            'learnlet_synthesis_kwargs': {
                'res': False,
            },
            'n_scales': 5,
            'exact_reconstruction_weight': 0,
            'input_size': (None, None, 1),     
        },
        'run_id': 'learnlet_dynamic_ht_div2k_(0, 55)_1575898840',
        'epoch': 250,
    },
]
all_exact_recon_net_params = [
    {
        'name': 'learnlet_exact_reco',
        'init_function': learnlet,
        'run_params': {
            'denoising_activation': SoftThresholding(thresh),
            'learnlet_analysis_kwargs':{
                'n_tiling': 256, 
                'mixing_details': False,        
            },
            'learnlet_synthesis_kwargs':{
            },
            'n_scales': 5,
            'exact_reconstruction_weight': 1,
            'input_size': (size, size, n_channels),     
        },
        'run_id': 'learnlet_div2k_30_1575052510',
        'epoch': 250,
    },
]

In [8]:
def unpack_model(init_function=None, run_params=None, run_id=None, epoch=250, **dummy_kwargs):
    model = init_function(**run_params)
    chkpt_path = f'checkpoints/{run_id}-{epoch}.hdf5'
    model.load_weights(chkpt_path)
    return model

def enumerate_seq(seq, name):
    return (seq[i] for i in tqdm_notebook(range(len(seq)), desc=f'Val files for {name}'))

def enumerate_seq_noisy(seq, name):
    return (np.squeeze(seq[i][0]) for i in tqdm_notebook(range(len(seq)), desc=f'Val files for {name}'))

def enumerate_seq_gt(seq):
    return (np.squeeze(seq[i][1]) for i in range(len(seq)))

def metrics_for_params(reco_function=None, name=None, **net_params):
    model = unpack_model(**net_params)
    metrics = Metrics()
    pred_and_gt = [
        (model.predict_on_batch(images_noisy), images_gt)
        for images_noisy, images_gt in enumerate_seq(im_bsd68, name)
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc=f'Stats for {name}'):
        metrics.push(images, im_recos.numpy())
    return metrics

def metrics_exact_recon_net(name=None, **net_params):
    model = unpack_model(**net_params)
    metrics = Metrics()
    pred_and_gt = [
        (model.predict_on_batch((images_noisy, images_noisy))[0], images_gt)
        for images_noisy, images_gt in enumerate_seq(im_bsd68, name)
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc=f'Stats for {name}'):
        metrics.push(images, im_recos.numpy())
    return metrics

def metrics_dynamic_denoising_net(name=None, **net_params):
    model = unpack_model(**net_params)
    metrics = Metrics()
    pred_and_gt = [
        (model.predict_on_batch((images_noisy, np.array([[noise_std/255]]))), images_gt)
        for images_noisy, images_gt in enumerate_seq(im_bsd68, name)
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc=f'Stats for {name}'):
        metrics.push(images, im_recos.numpy())
    return metrics

def metrics_original():
    metrics = Metrics()
    pred_and_gt = [
        (images_noisy, images_gt)
        for images_noisy, images_gt in enumerate_seq(im_bsd68, 'Original noisy image')
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc='Original noisy image'):
        metrics.push(images, im_recos)
    return metrics

def metrics_wavelets(wavelet_id):
    metrics = Metrics()
    pred = wavelet_denoising_pysap(
        enumerate_seq_noisy(im_bsd68, f'Wavelet denoising {wavelet_id}'),
        noise_std=noise_std/255,
        wavelet_id=wavelet_id, 
        n_scales=5, 
        soft_thresh=False, 
        n_sigma=3,
    )
    gt = enumerate_seq_gt(im_bsd68)
    for im_recos, images in tqdm_notebook(zip(pred, gt), desc='Stats for wavelet denoising'):
        metrics.push(images[..., None], im_recos[..., None])
    return metrics

def metrics_bm3d():
    metrics = Metrics()
    pred = [
        bm3d.bm3d(image_noisy + 0.5, sigma_psd=noise_std/255, stage_arg=bm3d.BM3DStages.ALL_STAGES) - 0.5
        for image_noisy in enumerate_seq_noisy(im_bsd68, f'BM3D')
    ]
    gt = enumerate_seq_gt(im_bsd68)
    for im_recos, images in tqdm_notebook(zip(pred, gt), desc='Stats for bm3d'):
        metrics.push(images[..., None], im_recos[..., None])
    return metrics

In [9]:
%%time
metrics = []
for net_params in all_net_params:
    metrics.append((net_params['name'], metrics_for_params(**net_params)))
    
for net_params in all_exact_recon_net_params:
    metrics.append((net_params['name'], metrics_exact_recon_net(**net_params)))
    
for net_params in dynamic_denoising_net_params:
    metrics.append((net_params['name'], metrics_dynamic_denoising_net(**net_params)))
    
metrics.append(('original', metrics_original()))
# metrics.append(('bm3d', metrics_bm3d()))
# metrics.append(('wavelets_24', metrics_wavelets('24')))


> <ipython-input-8-5f79d012d158>(41)metrics_dynamic_denoising_net()
     40     import ipdb; ipdb.set_trace()
---> 41     metrics = Metrics()
     42     pred_and_gt = [

ipdb> c



> <ipython-input-8-5f79d012d158>(41)metrics_dynamic_denoising_net()
     40     import ipdb; ipdb.set_trace()
---> 41     metrics = Metrics()
     42     pred_and_gt = [

ipdb> c



> <ipython-input-8-5f79d012d158>(41)metrics_dynamic_denoising_net()
     40     import ipdb; ipdb.set_trace()
---> 41     metrics = Metrics()
     42     pred_and_gt = [

ipdb> c



> <ipython-input-8-5f79d012d158>(41)metrics_dynamic_denoising_net()
     40     import ipdb; ipdb.set_trace()
---> 41     metrics = Metrics()
     42     pred_and_gt = [

ipdb> c



CPU times: user 32.6 s, sys: 2.09 s, total: 34.6 s
Wall time: 1min 15s


In [10]:
metrics

[('learnlet_exact_reco', PSNR = 27.08 +/- 4.704 SSIM = 0.7153 +/- 0.1317),
 ('learnlet_0_55', PSNR = 27.11 +/- 4.702 SSIM = 0.718 +/- 0.1354),
 ('learnlet_0_55_alpha_learned',
  PSNR = 27.16 +/- 4.619 SSIM = 0.7183 +/- 0.1298),
 ('learnlet_0_55_alpha_learned_bsd',
  PSNR = 27.11 +/- 4.396 SSIM = 0.7129 +/- 0.1174),
 ('learnlet_0_55_ht', PSNR = 26.9 +/- 4.613 SSIM = 0.7112 +/- 0.1401),
 ('original', PSNR = 18.59 +/- 0.03542 SSIM = 0.3283 +/- 0.2403)]

In [11]:
metrics.sort(key=lambda x: x[1].metrics['PSNR'].mean())

In [12]:
# import pickle
# with open('metrics_net_rec_fastmri', 'wb') as f:
#     pickle.dump(metrics, f)

In [13]:
def n_model_params_for_params(reco_function=None, val_gen=None, name=None, **net_params):
    model = unpack_model(**net_params)
    n_params = model.count_params()
    return n_params

In [14]:
%%time
n_params = {}
for net_params in all_net_params:
    n_params[net_params['name']] =  n_model_params_for_params(**net_params)
    
for net_params in dynamic_denoising_net_params:
    n_params[net_params['name']] =  n_model_params_for_params(**net_params)

for net_params in all_exact_recon_net_params:
    n_params[net_params['name']] =  n_model_params_for_params(**net_params)
    
n_params['original'] =  0
# n_params['bm3d'] =  2  # number of scales and type of wavelets
# n_params['wavelets_24'] =  2  # number of scales and type of wavelets

CPU times: user 2.41 s, sys: 104 ms, total: 2.51 s
Wall time: 2.43 s


In [15]:
def runtime_for_params(reco_function=None, name=None, **net_params):
    model = unpack_model(**net_params)
    data = im_bsd68[0]
    start = time.time()
    model.predict_on_batch(data[0])
    end = time.time()
    return end - start

def runtime_exact_recon_for_params(reco_function=None, name=None, **net_params):
    model = unpack_model(**net_params)
    data = im_bsd68[0]
    start = time.time()
    model.predict_on_batch((data[0], data[0]))
    end = time.time()
    return end - start

def runtime_dynamic_denoising_for_params(reco_function=None, name=None, **net_params):
    model = unpack_model(**net_params)
    data = im_bsd68[0]
    start = time.time()
    model.predict_on_batch((data[0], np.array([[noise_std/255]])))
    end = time.time()
    return end - start

In [16]:
%%time
runtimes = {}
for net_params in tqdm_notebook(all_net_params):
    runtimes[net_params['name']] =  runtime_for_params(**net_params)
    
for net_params in tqdm_notebook(all_exact_recon_net_params):
    runtimes[net_params['name']] =  runtime_exact_recon_for_params(**net_params)
    
for net_params in tqdm_notebook(dynamic_denoising_net_params):
    runtimes[net_params['name']] =  runtime_dynamic_denoising_for_params(**net_params)
    
runtimes['original'] = 'NA'
# runtimes['bm3d'] = 'NA'  # TODO: code function for that
# runtimes['wavelets_24'] = 'NA'  # TODO: code function for that


CPU times: user 2.73 s, sys: 59.4 ms, total: 2.79 s
Wall time: 2.73 s


In [17]:
metrics_table = pd.DataFrame(
    index=[name for name, _ in metrics], 
    columns=['PSNR-mean (std) (dB)', 'SSIM-mean (std)', '# params', 'Runtime (s)'],
)
for name, m in metrics:
    metrics_table.loc[name, 'PSNR-mean (std) (dB)'] = "{mean:.4} ({std:.4})".format(
        mean=m.metrics['PSNR'].mean(), 
        std=m.metrics['PSNR'].stddev(),
    )
    metrics_table.loc[name, 'SSIM-mean (std)'] = "{mean:.4} ({std:.4})".format(
        mean=m.metrics['SSIM'].mean(), 
        std=m.metrics['SSIM'].stddev(),
    )
    metrics_table.loc[name, '# params'] = "{}".format(
        n_params[name], 
     )
    metrics_table.loc[name, 'Runtime (s)'] = "{runtime:.4}".format(
        runtime=runtimes[name], 
    )

In [18]:
metrics_table

,PSNR-mean (std) (dB),SSIM-mean (std),# params,Runtime (s)
original,18.59 (0.01771),0.3283 (0.1202),0,NA
learnlet_0_55_ht,26.9 (2.306),0.7112 (0.07007),65456,0.05246
learnlet_exact_reco,27.08 (2.352),0.7153 (0.06584),65455,0.09662
learnlet_0_55_alpha_learned_bsd,27.11 (2.198),0.7129 (0.05869),65456,0.03626
learnlet_0_55,27.11 (2.351),0.718 (0.06769),65456,0.04766
learnlet_0_55_alpha_learned,27.16 (2.31),0.7183 (0.06488),65456,0.04564
